# Chapter 3 Multiple Linear Regression

edition 2
特征数据归一化
多元线性回归

## 导入相关库 

In [ ]:
###Boston房价预测模型
#导入模块

%matplotlib notebook 
import tensorflow as tf 
import pandas as pd 
import numpy as np
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

## 通过pandas导入数据 

In [ ]:

#读取数据文件

df = pd.read_csv("F:\Jupyter/tensorflow/MLR/data/boston.csv",header = 0)

#显示数据摘要描述信息
print(df.describe())


## 数据准备

In [ ]:
#获取df的值
df = df.values

print(df)

In [ ]:
#把 values格式转换成numpy的数组

df = np.array(df)
print(df)

## 数据特征归一化

In [ ]:
# 对特征数据【0到11】列做（0-1）的归一化

for i in range(12):
    df[:, i] = (df[:, i] - df[:, i].min()) /(df[:, i].max() - df[:, i].min())

In [ ]:
# x_data 为前12列的特征数据

x_data = df[:,:12]
# y_data 为最后一列标签数据
y_data = df[:,12]

In [ ]:
print(x_data, '\n shape=', x_data.shape)

In [ ]:
print(y_data, '\n shape=', y_data.shape)

## 模型建立

### 定义特征数据和标签数据的占位符

In [ ]:
#12个特征数据（12列）None表示行数不定，列数为12

x = tf.placeholder(tf.float32, [None, 12], name = "X")

#1个标签数据
y = tf.placeholder(tf.float32, [None, 1], name = "Y")

### 定义模型函数

In [ ]:
#定义一个命名空间
with tf.name_scope("Model"):
    w = tf.Variable(tf.random_normal([12, 1], stddev = 0.01), name = "W")
    b = tf.Variable(1.0, name = "b")
    def model(x,w,b):
        return tf.matmul(x, w) + b
    pred = model(x, w, b)
    
    
    
#给模型下的语句定义了一个命名空间，相当于打包在一起，计算图中会显示一个节点


## 3 训练模型 

### 设置模型训练超参数 

In [ ]:
train_epochs = 50
learning_rate = 0.01

### 定义损失函数&优化器

In [ ]:
with tf.name_scope("LossFunction"):
    loss_function = tf.reduce_mean(tf.pow(y-pred, 2))
    
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_function)

In [ ]:
###定义会话，初始化变量

sess = tf.Session()

init = tf.global_variables_initializer()
sess.run(init)

### 迭代训练

In [ ]:
for epoch in range(train_epochs):
    loss_sum = 0.0
    for xs, ys in zip(x_data, y_data):
        xs = xs.reshape(1 ,12)
        ys = ys.reshape(1, 1)
        
        _, loss = sess.run([optimizer, loss_function], feed_dict={x: xs,y: ys})
        loss_sum = loss_sum + loss
    x_data,y_data = shuffle(x_data, y_data)
    
    b0temp = b.eval(session=sess)
    w0temp = w.eval(session=sess)
    loss_average = loss_sum/len(y_data)
    print("epoch=", epoch+1, "loss=", loss_average, "b=", b0temp, "w=", w0temp)
    

##  模型预测结果

In [ ]:
n = np.random.randint(506) #随机选择一组数据来验证
print(n)
x_test = x_data[n]

x_test = x_test.reshape(1,12)     #此处的x_test一定不能为x_data，
                                  #每次x_data进行一次x_data[n]的操作后，
                                  #会成为一个一行12列的数组
predict = sess.run(pred, feed_dict={x:x_test})

print("预测值：%f" %predict)
target = y_data[n]
print("实际值：%f" %target)

error = abs(predict-target)/target
print("误差：%f" %error)